# Applied Seismology, GEOS 626, University of Alaska Fairbanks

- script hw_sumatraB_surf_lf.ipynb
- template script for analyzing surface wave amplitudes
- channel LHZ, duration up to 5 hours

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt 
import numpy as np
import os
import warnings

from lib_seis import get_dist_az
from lib_seis import smooth
from lib_seis import station_map_and_table
from lib_seis import sumatra_event
from obspy import read
from obspy import Stream
from pysep import Pysep
from pysep.recsec import plotw_rs
from scipy.signal import hilbert

In [ ]:
warnings.filterwarnings('ignore')                  
plt.rcParams['figure.figsize'] = 10, 5
plt.rcParams['lines.linewidth'] = 1

class SEED_ID_Error(Exception):
    pass

### Event details
Mw 9.1 2004 Sumatra - Andaman Islands Earthquake \
https://earthquake.usgs.gov/earthquakes/eventpage/official20041226005853450_30

In [ ]:
# fetch and display event details

event = sumatra_event()

print()
for i, item in enumerate(event):
    print(f'{item} = {event[item]}')
print()

In [ ]:
# download LHZ data, with response removed to your chosen output_unit
# choose your own inputs below

stations   = dict( networks                     = 'G,II,IU',
                   stations                     = '*',
                   locations                    = '*',
                   channels                     = 'LHZ' )

screening  = dict( mindistance_km               = 0 / 180 * np.pi * 6371,
                   maxdistance_km               = 40 / 180 * np.pi * 6371,
                   seconds_before_ref           = 1 * 60 * 60,
                   seconds_after_ref            = 4 * 60 * 60,
                   remove_clipped               = True,
                   remove_insufficient_length   = True,
                   fill_data_gaps               = 0,
                   gap_fraction                 = 0.03,
                   resample_freq                = 1     )

processing = dict( remove_response              = True,
                   output_unit                  = 'VEL',
                   water_level                  = None,
                   pre_filt                     = 'default',
                   detrend                      = True,
                   demean                       = True,
                   taper_percentage             = 0.2   )

output     = dict( log_level                    = 'INFO',
                   plot_files                   = 'map',
                   output_dir                   = 'datawf',
                   sac_subdir                   = '',
                   overwrite_event_tag          = 'sumatra_LHZ_vel',
                   overwrite                    = False )

In [ ]:
# download data

data_dir  = f'{output["output_dir"]}/{output["overwrite_event_tag"]}'
overwrite = f'{output["overwrite"]}'

if (not os.path.isdir(data_dir)) or (overwrite == 'True'):
    ps = Pysep(**event,**stations,**screening,**processing,**output)
    ps.run()
else:
    print('\nData directory already exists, no data will be downloaded\n')

In [ ]:
# read downloaded LHZ data

st = read(f'{data_dir}/*.sac', header=None)

print('Details of downloaded waveforms - \n')
print(st.__str__(extended=True))
print()

In [ ]:
# station map and distances table for the selected station locations
event_path = f'{data_dir}/event.xml'
station_map_and_table(st, event_path)

In [ ]:
# plot the record section using Pyseps's record section plotting tool

plot_info = dict( st                           = st,
                  sort_by                      = 'distance',
                  scale_by                     = 'normalize',
                  min_period_s                 = 1,
                  max_period_s                 = 10,
                  preprocess                   = 'st',
                  max_traces_per_rs            = 40,
                  distance_units               = 'deg',
                  tmarks                       = [0],
                  save                         = '',
                  log_level                    = 'DEBUG')

plotw_rs(**plot_info)

In [ ]:
# selecting waveforms with high frequency post-rupture signals

# modify the following list of picked waveforms -
picked_waveforms = ['IU.TATO.00.LHZ',
                    'II.PALK.00.LHZ',
                    'II.COCO.00.LHZ']

st_select = Stream()

for seed_id in picked_waveforms:
    st_pick = st.select(id=seed_id)
    if not st_pick:
        raise SEED_ID_Error(f'{seed_id} not in inventory')
    st_select += st_pick

In [ ]:
# plot the record section using Pyseps's record section plotting tool

plot_info = dict( st                           = st_select,
                  sort_by                      = 'distance',
                  scale_by                     = 'normalize',
                  min_period_s                 = 1,
                  max_period_s                 = 10,
                  preprocess                   = 'st',
                  max_traces_per_rs            = 40,
                  distance_units               = 'deg',
                  tmarks                       = [0],
                  save                         = '',
                  log_level                    = 'DEBUG')

plotw_rs(**plot_info)
